<a href="https://colab.research.google.com/github/aleanaliaaustral-web/labo2025v/blob/main/610_WorkFlow_01_gerencial_5sem_14_11_2025_EC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Nov 14 03:05:55 PM 2025"

In [2]:
rm(list = ls()) # Borrar todos los objetos
gc(verbose= FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660366,35.3,1454474,77.7,1454474,77.7
Vcells,1227093,9.4,8388608,64.0,1975136,15.1


cargo la libreria

In [3]:
# Instalar todas las dependencias necesarias para mlrMBO
install.packages("ParamHelpers")
install.packages("smoof")
install.packages("mlr")
install.packages("mlrMBO")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘BBmisc’, ‘checkmate’, ‘fastmatch’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RcppArmadillo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘parallelMap’, ‘XML’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘lhs’




In [4]:
# Instalar lightgbm desde CRAN
install.packages("lightgbm", repos = "https://cran.r-project.org")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
# Instalar DiceKriging y otras dependencias necesarias
install.packages("DiceKriging")
install.packages("rgenoud")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
require("data.table")
require("lightgbm")
require("mlrMBO")
require("mlrMBO")
require("ParamHelpers")  # <- Agregá esta línea

library(data.table)
library(lightgbm)
library(ParamHelpers)
library(mlrMBO)

Loading required package: data.table

Loading required package: lightgbm

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate



In [10]:
# NO OLVIDAR REEMPLAZAR SEMILLA PRIMIGENIA!
# Y EL NOMBRE DEL EXPERIMENTO!!

PARAM <- list()

PARAM$experimento <- "WF61002" # cambiar nombre
PARAM$semilla_primigenia <- 858956 # reemplazar por SU semilla primigenia

# establecer el Working Directory por defecto
setwd("/content/buckets/b1/exp/")

In [11]:
# creo la carpeta para este experimento
dir.create(paste0(PARAM$experimento, "/"), showWarnings= FALSE)
setwd(paste0(PARAM$experimento, "/")) # Establezco el Working Directory DEL EXPERIMENTO

setwd(paste0("./"))

In [12]:
# Instalar R.utils si no está disponible
if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:smoof’:

    getDescription, getName


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC

#### Data Loading

In [13]:
dataset <- fread("/content/datasets/gerencial_competencia_2025.csv.gz")

#### Data Drifting

In [14]:
# En esta semilla, NO hay data drifting
# los alumnos que tengan su dataset SIN data drifting, deben cambiar este parametro a FALSE

PARAM$drifting$hay_drifting <- TRUE

if( PARAM$drifting$hay_drifting ) {
  dataset[foto_mes == 202107, clase_ternaria := ifelse(clase_ternaria == "CONTINUA", "BAJA+2", clase_ternaria)]
}

#### Training Strategy

Establezco los periodos que voy a usar para train y test

In [15]:
# la training strategy del ejemplo del notebook
# no es la mejor, deben reemplazarlo por la que Uds encontraron,
# donde SEGURO hicieron un trabajo EXCELENTE con el script  z211_creacion_TS.r
# ¿ verdad que ese fue el caso ?

PARAM$trainingstrategy$training <- c(202103, 202104, 202105)
PARAM$trainingstrategy$validation <- c(202107)
PARAM$trainingstrategy$testing <- c(202109)
PARAM$trainingstrategy$final_train <- c(202103, 202104, 202105, 202107)
PARAM$trainingstrategy$future <- c(202109)

#### Feature Engineering

Voy a crear 60 lagueados estadisticos

In [16]:
cols_lagueables <- copy(setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# ordeno de esta forma por el concepto de Tempus Fugit
setorder(dataset, numero_de_cliente, foto_mes)

In [17]:
PARAM$lag$meses <- 2 # 2 meses de lag
PARAM$lag$deltas <- TRUE # con tendencia
PARAM$lag$delta_pct <- FALSE # sin tendencia porcentual

In [18]:
# creo los campos lags de orden 1 y 2
# como hay que ordenar por { numero_de_cliente, foto_mes }
# debo utilizar Rcpp

for (vcol in cols_lagueables) {
  for (im in 1:PARAM$lag$meses) {
    dataset[, paste0(vcol, "_lag", im) :=
      shift(get(vcol), n = im, type = "lag"),
    by = numero_de_cliente]
  }
}

if( PARAM$lag$deltas ){
  # diferencias con lags en forma NO porcentual
  for (vcol in cols_lagueables) {
    for (im in 1:PARAM$lag$meses) {
      dataset[, paste0(vcol, "_delta", im) :=
        get(vcol) - get(paste0(vcol, "_lag", im))]
    }
  }
}

if( PARAM$lag$delta_pct ){
  # diferencias con lags en forma porcentual
  for (vcol in cols_lagueables) {
    for (im in 1:PARAM$lag$meses) {
      dataset[, paste0(vcol, "_delta", im, "_pct") :=
        get(paste0(vcol, "_delta", im)) /  (get(paste0(vcol, "_lag", im)) + 0.0001)]
    }
  }
}

In [19]:
# defino los nombres de los campos
campos_buenos <- setdiff(colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria"))

#### Bayesian Optimization - Hiperparametros

**AQUÍ ESTÁ LA CORRECCIÓN DEL BUG**

Aqui se deben agregar los hiperparametros que quieren optimizar

In [20]:
# Aqui se cargan los hiperparametros

PARAM$bo_lgb <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.2),
  makeNumericParam("feature_fraction", lower = 0.3, upper = 1.0),
  makeIntegerParam("min_data_in_leaf", lower = 0L, upper = 4000L),
  makeIntegerParam("num_leaves", lower = 8L, upper = 1024L),
  makeNumericParam("lambda_l1", lower = 0.0, upper = 10.0),
  makeNumericParam("lambda_l2", lower = 0.0, upper = 10.0)
)

# Aqui la cantidad de iteraciones de la Bayesian Optimization
PARAM$bo_iteraciones <- 50

# Ahora defino el controlador para pasar por las 5 semillas
# Uso 1 semilla original + 4 semillas adicionales generadas aleatoriamente

PARAM$semillas <- c(PARAM$semilla_primigenia)
set.seed(PARAM$semilla_primigenia)
for(i in 1:4) {
  PARAM$semillas <- c(PARAM$semillas, sample.int(1000000, 1))
}

In [21]:
# AQUÍ ESTÁ LA CORRECCIÓN PRINCIPAL:
# num_iterations y early_stopping_rounds NO van como extras, van en el parámetro fijos

PARAM$out <- list()
PARAM$out$lgbm <- list()
PARAM$out$lgbm$mejor_ganancia <- -Inf

# funcion que optimiza con 5 semillas distintas y toma el promedio
# Recibe hiperparametros de LGBM
fganancia_lgbm_meseta <- function(x) {
  # inicializo en cero los vectores para acumular ganancias
  ganancias <- numeric(length(PARAM$semillas))

  # para cada semilla
  for(idx_semilla in 1:length(PARAM$semillas)) {
    semilla_actual <- PARAM$semillas[idx_semilla]

    # defino los parametros COMPLETOS
    # CORRECCIÓN: num_iterations y early_stopping_rounds van en param_train
    param_train <- list(
      objective = "binary",
      metric = "custom",
      first_metric_only = TRUE,
      boost_from_average = TRUE,
      feature_pre_filter = FALSE,
      num_iterations = 9999,  # MOVIDO AQUÍ
      early_stopping_rounds = 200,  # MOVIDO AQUÍ
      verbosity = -100,
      seed = semilla_actual,
      max_depth = -1,
      min_gain_to_split = 0.0,
      min_sum_hessian_in_leaf = 0.001,
      lambda_l1 = x$lambda_l1,
      lambda_l2 = x$lambda_l2,
      learning_rate = x$learning_rate,
      feature_fraction = x$feature_fraction,
      min_data_in_leaf = x$min_data_in_leaf,
      num_leaves = x$num_leaves
    )

    # genero el modelo con esta semilla
    modelo <- lgb.train(
      data = dtrain,
      valids = list(valid = dvalidate),
      eval = lgb_gan_eval,
      param = param_train  # Ya incluye todo lo necesario
    )

    # capturo la ganancia
    ganancias[idx_semilla] <- unlist(modelo$record_evals$valid$ganancia$eval)[[modelo$best_iter]]
  }

  # calculo ganancia promedio
  ganancia_promedio <- mean(ganancias)
  ganancia_promedio_normalizada <- ganancia_promedio * 0.3

  # si esta ganancia es mejor que la mejor hasta ahora, la guardo
  if (ganancia_promedio_normalizada > PARAM$out$lgbm$mejor_ganancia) {
    PARAM$out$lgbm$mejor_ganancia <<- ganancia_promedio_normalizada
    PARAM$out$lgbm$mejores_hiperparametros <<- as.list(x)
    PARAM$out$lgbm$mejor_iteracion <<- modelo$best_iter
  }

  return(ganancia_promedio_normalizada)
}

In [22]:
# creo los datasets de training y validation
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$training, campos_buenos, with = FALSE]),
  label = dataset[foto_mes %in% PARAM$trainingstrategy$training, ifelse(clase_ternaria == "BAJA+2", 1L, 0L)],
  free_raw_data = FALSE
)

dvalidate <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validation, campos_buenos, with = FALSE]),
  label = dataset[foto_mes %in% PARAM$trainingstrategy$validation, ifelse(clase_ternaria == "BAJA+2", 1L, 0L)],
  free_raw_data = FALSE
)

In [23]:
# Defino la metrica customizada ganancia usando la clase binaria BAJA+2
lgb_gan_eval <- function(y_pred, dtrain) {
  # Obtener labels usando el método correcto para lgb.Dataset
  labels <- dtrain$get_field("label")

  # vector de ganancias
  gan <- ifelse(labels == 1 & y_pred >= 0.025, 273000,
    ifelse(labels == 0 & y_pred >= 0.025, -7000, 0)
  )

  # esta es la ganancia
  ganancia <- sum(gan)

  return(list(
    name = "ganancia",
    value = ganancia,
    higher_better = TRUE
  ))
}

In [25]:
# Aqui comienza la configuracion de la Bayesian Optimization
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana, los hiperparametros que se van a optimizar
obj.fun <- makeSingleObjectiveFunction(
  name = "lgbm_ganancia",
  fn = fganancia_lgbm_meseta,
  par.set = PARAM$bo_lgb,
  has.simple.signature = FALSE,
  minimize = FALSE  # <-- cambiar 'minimization' por 'minimize'
)

ctrl <- makeMBOControl()
ctrl <- setMBOControlTermination(ctrl, iters = PARAM$bo_iteraciones)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# inicio la optimizacion bayesiana
if (!file.exists("bayesiana.RDATA")) {
  run <- mbo(obj.fun, control = ctrl)

  # salvo a disco el resultado
  save(run, file = "bayesiana.RDATA")
} else {
  # cargo el resultado
  load(file = "bayesiana.RDATA")
}

#### Consolidacion
Ahora voy a unir los mejores hiperparametros encontrados en la Bayesian Optimizacion con los hiperparametros que no optimice

In [26]:
fijos <- list(
  objective = "binary",
  metric = "custom",
  first_metric_only = TRUE,
  boost_from_average = TRUE,
  feature_pre_filter = FALSE,
  verbosity = -100,
  max_depth = -1,
  min_gain_to_split = 0.0,
  min_sum_hessian_in_leaf = 0.001,
  seed = PARAM$semilla_primigenia,
  num_iterations = PARAM$out$lgbm$mejor_iteracion
)

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [27]:
# creo dataset final_train con todos los meses de entrenamiento
dfinal_train <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$final_train, campos_buenos, with = FALSE]),
  label = dataset[foto_mes %in% PARAM$trainingstrategy$final_train, ifelse(clase_ternaria == "BAJA+2", 1L, 0L)],
  free_raw_data = FALSE
)

final_model <- lgb.train(
  data = dfinal_train,
  param = param_final,
  verbose = -100
)

In [28]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [29]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [30]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[foto_mes %in% PARAM$trainingstrategy$future]

In [31]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)

##### Tabla Prediccion

In [32]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file = "prediccion.txt",
  sep = "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [33]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste("-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
    "  semilla=", PARAM$semilla_primigenia,
    "'")

  linea <- paste(comando, competencia, arch, mensaje)

  salida <- system(linea, intern = TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 


In [34]:
# grabo los parametros
if (!require("yaml")) install.packages("yaml")
require("yaml")

write_yaml(PARAM, file = "PARAM.yml")

Loading required package: yaml



In [35]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Nov 14 06:31:56 PM 2025"

In [36]:
# Ver los mejores resultados
print("Mejor ganancia encontrada:")
print(PARAM$out$lgbm$mejor_ganancia)

print("Mejores hiperparámetros:")
print(PARAM$out$lgbm$mejores_hiperparametros)

print("Mejor iteración:")
print(PARAM$out$lgbm$mejor_iteracion)

[1] "Mejor ganancia encontrada:"
[1] 53266920
[1] "Mejores hiperparámetros:"
$learning_rate
[1] 0.1960674

$feature_fraction
[1] 0.5883752

$min_data_in_leaf
[1] 4000

$num_leaves
[1] 118

$lambda_l1
[1] 1.768192

$lambda_l2
[1] 0.02043866

[1] "Mejor iteración:"
[1] 30


In [37]:
# Ver el resumen de la optimización
print(run)
run$y  # mejor valor objetivo encontrado
run$x  # mejores hiperparámetros

Recommended parameters:
learning_rate=0.196; feature_fraction=0.588; min_data_in_leaf=4000; num_leaves=118; lambda_l1=1.77; lambda_l2=0.0204
Objective: y = 53266920.000

Optimization path
24 + 50 entries in total, displaying last 10 (or less):
   learning_rate feature_fraction min_data_in_leaf num_leaves  lambda_l1
65    0.19971910        0.6882920             2239        211 9.99739517
66    0.01004666        0.9984138             2092        330 4.05750771
67    0.01015767        0.3082793             3926       1024 1.79100772
68    0.19998969        0.3201362             4000        130 1.30103373
69    0.01086671        0.5030037             3611       1024 9.98925220
70    0.18139706        0.9957262             2903        385 0.96719662
71    0.19992966        0.5550632             3997        383 1.66619878
72    0.19980998        0.4189537             3997        368 0.93729420
73    0.02425011        0.3001852             3931         93 0.01571047
74    0.13083478        0.

[1] 53266920

$learning_rate
[1] 0.1960674

$feature_fraction
[1] 0.5883752

$min_data_in_leaf
[1] 4000

$num_leaves
[1] 118

$lambda_l1
[1] 1.768192

$lambda_l2
[1] 0.02043866